# **The Quest to fill NaN's**


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import gc
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
#Standard (memory efficent) dtype conversion for Microsoft Malware Prediction dataset 

dtypes = {
        'MachineIdentifier':                                    'str',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float16',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int8',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float32',
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float16',
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float16',
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float32',
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float32',
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float16',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float16',
        'Census_InternalPrimaryDisplayResolutionVertical':      'float16',
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float32',
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }

# Part I: NaN killed the Feature
There are plenty of kernels detailing various reasons to drop features i.e. 'random' label ('MachineIdentifier'), large (>50%) NaN counts, low (<1%) variance. 

In [ ]:
# The columns which cause more problems than they solve
drop_columns = ['MachineIdentifier',
                'PuaMode',
                'Census_IsFlightingInternal',
                'Census_InternalBatteryType',
                'Census_ProcessorClass',
                'Census_IsFlightsDisabled',
                'Census_ThresholdOptIn',
                'Census_IsWIMBootEnabled',
                'SMode',
                "AutoSampleOptIn",
                "IsBeta",
                "Census_IsPortableOperatingSystem"]

Dropping features is a last resort. In doing so, it is always good practice to make underlying assumptions explicit. For instance, what do you think when you see,

'**SMode':**
* 6% of its values missing while **of the remaining non-NaN values** 99.96% are '0' and 0.04% are '1' meaning  'only Microsoft Store apps can be installed'.
* Of the 0.04% systems, 17% of them 'HasDetection' == 1.

You might see that 17% and think, 'Hey, that's really low, we can use it!". First we have to fill the NaN's. If we fill the NaN's we risk more in terms of bias than we stand to gain in terms of accuracy.
If you wish to keep this feature (for that 0.04%), Mode imputation or Last-Observation-Carried-Forward are fine since these would propogate '0' for the most part, meaning it would identify NaN's as machines 
for which **not** only Microsoft Store apps can be installed'


In [ ]:
usecols = [column for column in dtypes.keys() if column not in drop_columns]
train = pd.read_csv('../input/train.csv', usecols = usecols, dtype=dtypes)
test = pd.read_csv('../input/test.csv', usecols = usecols[:-1], dtype=dtypes)

# An alternative is to load the data through Dask which runs in parallel on most reasonable function calls (to compile use .compute())
# import dask.dataframe as dd
# train = dd.read_csv('../input/train.csv', dtype=dtypes)
# test = dd.read_csv('../input/test.csv', dtype=dtypes)
# train, test = train[usecols], test[usecols[:-1]]


# Part 2. Who are the NaN? Where did they come from?

Let's explore the train data explicitly.

In [ ]:
# There are some features remaining which retain large NaN counts 
NaN_columns = [col for col in train.columns if train[col].isna().any()]
train_missing = train[NaN_columns]

# Missing data column : Percentage of missing values
(train_missing.isna().sum()/train_missing.isna().count()).sort_values(ascending=False).head(10)

Notice that 'DefaultBrowsersIdentifier', 'SmartScreen' and 'OrganizationIdentifier' have NaN percentages **>30%**. In particular, 'DefaultBrowsersIdentifier' has **>95%** missing values, hold on, we've dropped features for less in our earlier considerations. 

In [ ]:
import missingno as msno
#https://github.com/ResidentMario/missingno

#The msno.matrix nullity matrix is a data-dense display of nullity with visualization supporting up to 50 columns
msno.matrix(train[['DefaultBrowsersIdentifier']])

The y-axis is identified with the (natural) index.

Since the dataset provided here has been roughly split by time, we might be able to use regularities in 'DefaultBrowsersIdentifier' to identify time-splits. Let's see if we can find any regularities!

In [ ]:
import matplotlib.pyplot as plt

# We abuse the visualization scheme of missingno to identify occurence patterns in different values of 'DefaultBrowsersIdentifier'
# value     count
# 239.0     46056
# 3196.0    42694
# 1632.0    28751
# 3176.0    24222
# 146.0     20756

for value in train.DefaultBrowsersIdentifier.value_counts().index[:5]:
    print('Location of DefaultBrowsersIdentifier value {}'.format(value))
    msno.matrix(train[['DefaultBrowsersIdentifier']].fillna(0).replace({value:np.nan}))
    
# It is important to notice the last example, which has missing values not appearing on the visualization due to sparsity:
# plt.plot(train[['DefaultBrowsersIdentifier']].where(train.DefaultBrowsersIdentifier==146.0))

Thus we arrive at a cross-roads:
1. Drop 'DefaultBrowsersIdentifier'
2. Featurize time-split and drop 'DefaultBrowsersIdentifier'

We will choose (1) for the moment.

In [ ]:
del train['DefaultBrowsersIdentifier']
del test['DefaultBrowsersIdentifier']
gc.collect()

Where to go from here? Let's review the classification of NaN's.

There are several different classification of missingness patterns: 
* Missingness completely at random (MCAR),
* Missingness at random (MAR), 
* Missingness that depends on unobserved predictors (MNAR)

The missingno correlation heatmap measures nullity correlation: how strongly the presence or absence of one variable affects the presence of another:
* -1 (if one variable appears the other definitely does not)
* 0 (variables appearing or not appearing have no effect on one another)
* 1 (if one variable appears the other definitely also does).

In [ ]:
msno.heatmap(train_missing)

Looking at the NaN correlation map above,  
* 'IsProtected' and  'AVProductStatesIdentifier',  'AVProductsInstalled', ' 'AVProductsEnabled' have 1.0 correlation.
* 'AVProductStatesIdentifier' and 'AVProductsEnabled' have 1.0 correlation.
* 'Census_ProcessorModelIdentifier' and 'Census_ProcessorCoreCount' have 1.0 correlation. 

What does this mean for us when we consider NaN preprocessing techniques?

In [ ]:
# Another useful visualization of missingness is a Dendrogram
msno.dendrogram(train_missing)

To interpret this graph, read it from a top-down perspective.
* Cluster leaves which link together at a distance of zero fully predict one another's presence—one variable might always be empty when another is filled, or they might always both be filled or both empty, and so on. 

In this specific example the dendrogram identifies a strong missingness pattern between 'Wdft_IsGamer' and 'Wdft_RegionIdentifier' for instance.'

Let's take a look at SmartScreen, 'SmartScreen Filter helps you identify reported phishing and malware websites and also helps you make informed decisions about downloads.'
SmartScreen values can be interpreted to be of the following types
* RequireAdmin = Get administrator approval before running an unrecognized app from the internet:
* Prompt = Warn before running an unrecognized app, but don't require administrator approval
* Off = Don't do anything
* ExistsNotSet = Exists not set (maybe new machine or indicative of careless administrator?) 

Using what we've learned above won't apply here. Let's observe some properties of SmartScreen.

In [ ]:
import seaborn as sns

sns.countplot(x=train.SmartScreen, hue = train.HasDetections)
plt.xticks(rotation=90)
plt.title('HasDetection counts of SmartScreen values')
plt.legend(loc='upper right')
plt.show()

print('Total counts of SmartScreen values (excluding NaNs)')
train.SmartScreen.value_counts()

The named categorical data seems inconsistent at best, too many terms without representation. We replace values with a consistent heuristic (my judgement).

In [ ]:
SmartScreen_dict = {
 'RequireAdmin': 'RequireAdmin',
 'ExistsNotSet': 'ExistsNotSet',
 'Off': 'Off',
 'Warn': 'Prompt',
 'Prompt': 'Prompt',
 'Block': 'RequireAdmin',
 'off': 'Off',
 'On': 'RequireAdmin',
 'on': 'RequireAdmin',
 '&#x01;': 'Off',
 '&#x02;': 'Off',
 'OFF': 'Off',
 'Promt': 'Prompt',
 '0': 'Off',
 'requireadmin': 'RequireAdmin',
 'Enabled': 'RequireAdmin',
 'prompt': 'Prompt',
 'warn': 'Prompt',
 '00000000': 'Off',
 'requireAdmin': 'RequireAdmin',
 '&#x03;': 'Off',
 'Promprt': 'Prompt',
 'RequiredAdmin': 'RequireAdmin',
 'of': 'Off',
 'Deny': 'RequireAdmin',
 'ON': 'RequireAdmin',
 'BLOCK': 'RequireAdmin'
 }


sns.countplot(x=train.SmartScreen.replace(to_replace=SmartScreen_dict).fillna('NaN'), hue = train.HasDetections)
plt.xticks(rotation=90)
plt.legend(loc='upper right')
plt.title('HasDetection counts of SmartScreen values (including NaN)')
plt.show()

In [ ]:
# Looks good! Let's change the values in train in accordance with SmartScreen_dict
train.SmartScreen.replace(to_replace=SmartScreen_dict, inplace = True)

# Let's see if these values are skewed at all toward a particular value of 'HasDetections'
print('Ratio of HasDetection for SmartScreen values')
(train['HasDetections'].groupby(train['SmartScreen'].fillna('Missing')).value_counts()/train['HasDetections'].groupby(train['SmartScreen'].fillna('Missing')).count()).unstack().plot(kind = 'bar')

Why are we featurizing before imputing NaN's? 
* To assist in the classification of NaNs as MAR, MCAR or MNAR (for definition see above)
* To reduce number of bins of categorical (nominal) data

There is a question of combining 'Off' and 'Prompt' (explored via missingno.matrix exploration) but this goes against the technical definitions which I am presuming. 

Since the count of NaN values is greater than  50%, this feature leads us to our first major dilemma, what to do with the NaNs? If we simply replace them with 'Missing' we risk lossing data points which
could be categorized in 'ExistsNotSet' which is highly skewed towards 1 in 'HasDetections'.


# Part 3: The Quest to fill the NaN's

Let's recall the most common of all NaN imputations:

* Dummy-fication 



In [ ]:
print('Dummy-fication of SmartScreen:')
pd.concat([new_train[['SmartScreen']], new_train[['SmartScreen']].isnull().astype(int).add_suffix('_NAN')], axis = 1).head(10)

* Mean, Mode and Median imputation

In [ ]:
print('Mode imputation of SmartScreen:')
pd.concat([new_train[['SmartScreen']], new_train[['SmartScreen']].fillna(train.SmartScreen.mode().values[0])], axis = 1).head(10)

It's easy to overlook theses imputation methods as classical and prefer to work with more "advanced" techniques. However, if we leverage our understanding of the data structure, we might be able to do better than algorithmic imputation. 

Let's look at some canidates for such imputations, we will start with those categories with low missing NaN counts and conclude with 'SmartScreen'.


In [ ]:
(train_missing.isna().sum()/train_missing.isna().count()).sort_values(ascending=True).head(10)

In [ ]:
# Let's see if we can figure out how to fill in the missing values of OsBuildLab
train[['OsBuildLab', 'OsBuild', 'Census_OSBuildNumber', 'Processor', 'Census_OSArchitecture' , 'Census_OSBranch',  'OsPlatformSubRelease']].head(10)

Looking at the above tabular data, we are inclined to suggest the following psudo-equation for OsBuildLab:
*  **OsBuildLab = (OsBuild  =? Census_OSBuildNumber) + (Census_OSArchitecture =? Processor) + Census_OSBranch**



In [ ]:
# In fact, the above table is quite misleading, 

# To see that OsBuild != Census_OSBuildNumber:
# train[['OsBuild', 'Census_OSBuildNumber']].groupby('OsBuild')['Census_OSBuildNumber'].value_counts()

# Also, 'Census_OSArchitecture' != Processor
# train[['Census_OSArchitecture', 'Processor']].groupby('Processor')['Census_OSArchitecture'].value_counts()

# So which one do we use in our equation to represent OsBuildLab? 
# train[(train['Census_OSBuildNumber'] != 10586) & (train['OsBuild'] == 7601)][['OsBuildLab', 'OsBuild', 'Census_OSBuildNumber']]
# train[(train['Census_OSArchitecture'] != 'x64') & (train['Processor'] == 'x64')][['OsBuildLab', 'Processor', 'Census_OSArchitecture']] 
# train[(train['OsPlatformSubRelease'] != 'rs3') & (train['Census_OSBranch'] == 'rs3_release')][['OsBuildLab', 'OsPlatformSubRelease', 'Census_OSBranch']]
# A: OsBuild and Processor

print('Table of Missing values in OsBuildLab with most relevant data')
train[['OsBuildLab', 'OsBuild', 'Processor' , 'OsPlatformSubRelease']][train['OsBuildLab'].isna()]

In [ ]:
train[['OsBuildLab', 'OsBuild', 'Processor' , 'OsPlatformSubRelease']]

Let's see if we can find some good canidates to fill those missing values!

In [ ]:
# Let's see if we can find some good canidates to fill those missing values!
index_NaN = []
NaN_clues = [] # List of relevant values for re-constructing NaN's in OsBuildLab
for row in train.loc[train['OsBuildLab'].isna(),['OsBuild', 'Processor', 'OsPlatformSubRelease']].iterrows():
    index, data = row
    index_NaN.append(index)
    NaN_clues.append(data.tolist())

# To avoid changing data we create a copy of 'OsBuildLab'
OsBuildLab_NaN = train['OsBuildLab'].copy()
OsBuildLab_NaN = OsBuildLab_NaN.cat.add_categories(['Missing'])
OsBuildLab_NaN.fillna('Missing', inplace = True)

base = r'^{}'# Matches the start of a string
expr = '(?=.*{})' # Look ahead to match the string inside {}

fill_NaN = []
OsBuildLab_clean = []
for i, NaN_row in enumerate(NaN_clues):
    
    mask = OsBuildLab_NaN.str.contains(base.format(''.join(expr.format(str(w)[1:]) for w in NaN_row[:-1])), regex = True)
    print('Missing NaN value possibilities for Row %s : ' % i + str(NaN_row))
    print(OsBuildLab_NaN[mask].value_counts()[0:5])
    OsBuildLab_clean.append(OsBuildLab_NaN[mask].value_counts().index[0])
    fill_NaN.append(OsBuildLab_NaN[mask].value_counts().index[0])

Although only 7 of the missing values can be imputed based on highly skewed value counts, we will store the most likely (most frequent) option in a dataframe for comparison later.

Notice how we've discovered a linear relationship and analyzed several consequences relatively quickly. How would the next best thing, a multiple imputation algorithm do the job?

* **'Census_OSVersion' = 'OsVer ' + 'Census_OSBuildNumber' + 'Census_OSBuildRevision'**

In [ ]:
train[['OsBuild', 'Processor' , 'OsPlatformSubRelease']].isna().sum()

In [ ]:
train[['Census_OSVersion' ,'Census_OSBuildNumber', 'OsBuildLab', 'Census_OSBuildRevision']]

How do we know that we used the correct class among  'Census_OSBuildNumber' and OsBuild'? We check of course (i'll leave this to the skeptical reader). 

So what does this all mean? 

'We have two choices for **'SmartScreen'**
1. Dummy-fication. If the problem is challenging, take the simple solution.
2. Classification via training LogReg/NB/RF/SVM/kNN on non-NaN training set and apply classifier on NaN training set

I assume that NaN's in SmartScreen are Missing at Random, that is to say, "where missingness can be fully accounted for by variables where there is complete information" - wiki/Missing_data.